In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import clickhouse_connect
import os
from datetime import date

os.environ['NO_PROXY'] = '10.80.131.73'
connection = clickhouse_connect.get_client(host = "10.80.131.73", port = 8123, username = 'test', password = 'secretPassword321!', database = 'test')


allowed_val = ["Windows 10", 'Windows Server 2008','Windows 8', 'Windows 7', 'Windows Server 2012', 'Windows 11', 'Windows Server 2016', 'Windows Server 2019', 'Windows Server 2022', 'Windows Server']

rep_dict = {
            'Windows 10 for x64-based Systems': "Windows 10", 'Windows 10 for 32-bit Systems': "Windows 10", 'Windows 10 Version 22H2 for x64-based Systems': "Windows 10", 'Windows 10 Version 22H2 for ARM64-based Systems': "Windows 10", 
            'Windows 10 Version 22H2 for 32-bit Systems':'Windows 10',  "Windows 10 Version 21H2 for x64-based Systems": "Windows 10", 'Windows 10 Version 21H2 for ARM64-based Systems': "Windows 10", 'Windows 10 Version 21H2 for 32-bit Systems': "Windows 10", 
            'Windows 10 Version 21H1 for x64-based Systems': "Windows 10", 'Windows 10 Version 21H1 for ARM64-based Systems': "Windows 10", 'Windows 10 Version 21H1 for 32-bit Systems': "Windows 10", 'Windows 10 Version 20H2 for x64-based Systems': "Windows 10", 
            'Windows 10 Version 20H2 for ARM64-based Systems': "Windows 10", 'Windows 10 Version 20H2 for 32-bit Systems': "Windows 10", 'Windows 10 Version 2004 for x64-based Systems': "Windows 10", 'Windows 10 Version 2004 for ARM64-based Systems': "Windows 10", 
            'Windows 10 Version 2004 for 32-bit Systems': "Windows 10", 'Windows 10 Version 1909 for x64-based Systems': "Windows 10", 'Windows 10 Version 1909 for ARM64-based Systems': "Windows 10", 'Windows 10 Version 1909 for 32-bit Systems': "Windows 10", 
            'Windows 10 Version 1903 for x64-based Systems': "Windows 10", 'Windows 10 Version 1903 for ARM64-based Systems': "Windows 10", 'Windows 10 Version 1903 for 32-bit Systems': "Windows 10", 'Windows 10 Version 1809 for x64-based Systems': "Windows 10", 
            'Windows 10 Version 1809 for ARM64-based Systems': "Windows 10", 'Windows 10 Version 1809 for 32-bit Systems': "Windows 10", 'Windows 10 Version 1803 for x64-based Systems': "Windows 10", 'Windows 10 Version 1803 for ARM64-based Systems': "Windows 10", 
            'Windows 10 Version 1803 for 32-bit Systems': "Windows 10", 'Windows 10 Version 1709 for x64-based Systems': "Windows 10", 'Windows 10 Version 1709 for ARM64-based Systems': "Windows 10", 'Windows 10 Version 1709 for 32-bit Systems': "Windows 10",
            'Windows 10 Version 1607 for x64-based Systems': "Windows 10", 'Windows 10 Version 1607 for 32-bit Systems': "Windows 10", 
            'Microsoft Windows 10 Enterprise':'Windows 10', 'Microsoft Windows 10 Enterprise LTSC':'Windows 10', 'Microsoft Windows 10 Enterprise N 2016 LTSB':'Windows 10', 'Майкрософт Windows 10 Корпоративная':'Windows 10', 'Майкрософт Windows 10 Корпоративная LTSC':'Windows 10', 
            
            'Windows Server 2008 for x64-based Systems Service Pack 2 (Server Core installation)':'Windows Server 2008', 'Windows Server 2008 for x64-based Systems Service Pack 2':'Windows Server 2008', 'Windows Server 2008 for Itanium-Based Systems Service Pack 2':'Windows Server 2008',
            'Windows Server 2008 for 32-bit Systems Service Pack 2 (Server Core installation)':'Windows Server 2008', 'Windows Server 2008 for 32-bit Systems Service Pack 2':'Windows Server 2008', 'Windows Server 2008 R2 for x64-based Systems Service Pack 1 (Server Core installation)':'Windows Server 2008',
            'Windows Server 2008 R2 for x64-based Systems Service Pack 1':'Windows Server 2008', 'Windows Server 2008 R2 for Itanium-Based Systems Service Pack 1':'Windows Server 2008', 'Microsoft Windows Server 2008 R2 Standard': 'Windows Server 2008', 
            'Microsoft Windows Server 2008 R2 Enterprise': 'Windows Server 2008', 'Microsoft Windows Server 2008 R2 Enterprise ': 'Windows Server 2008', 'Microsoft Windows Server 2008 R2 Standard ': 'Windows Server 2008', 

            'Windows 8.1 for x64-based systems': 'Windows 8', 'Windows 8.1 for 32-bit systems': 'Windows 8', 

            'Windows 7 for x64-based Systems Service Pack 1': 'Windows 7', 'Windows 7 for 32-bit Systems Service Pack 1': 'Windows 7', 

            'Windows Server, version 20H2 (Server Core Installation)':'Windows Server', 'Windows Server, version 2004 (Server Core installation)':'Windows Server', 'Windows Server, version 1909 (Server Core installation)':'Windows Server', 
            'Windows Server, version 1903 (Server Core installation)':'Windows Server', 'Windows Server, version 1803 (Server Core Installation)':'Windows Server', 
    
            'Windows Server 2012 (Server Core installation)':'Windows Server 2012', 'Windows Server 2012 R2':'Windows Server 2012', 'Windows Server 2012 R2 (Server Core installation)':'Windows Server 2012', 'Microsoft Windows Server 2012 R2 Datacenter':'Windows Server 2012',
            'Microsoft Windows Server 2012 R2 Standard': 'Windows Server 2012',
            
            'Windows 11 Version 22H2 for ARM64-based Systems':'Windows 11', 'Windows 11 Version 22H2 for x64-based Systems':'Windows 11', 'Windows 11 Version 23H2 for ARM64-based Systems':'Windows 11', 
            'Windows 11 Version 23H2 for x64-based Systems':'Windows 11', 'Windows 11 Version 24H2 for ARM64-based Systems':'Windows 11', 'Windows 11 Version 24H2 for x64-based Systems':'Windows 11', 'Windows 11 version 21H2 for ARM64-based Systems':'Windows 11', 'Windows 11 version 21H2 for x64-based Systems':'Windows 11', 
            
            'Windows Server 2016 (Server Core installation)':'Windows Server 2016', 'Microsoft Windows Server 2016 Datacenter':'Windows Server 2016', 'Microsoft Windows Server 2016 Standard':'Windows Server 2016', 'Майкрософт Windows Server 2016 Datacenter':'Windows Server 2016',
            'Майкрософт Windows Server 2016 Standard':'Windows Server 2016',
            
            'Windows Server 2019 (Server Core installation)':'Windows Server 2019', 'Microsoft Windows Server 2019 Datacenter':'Windows Server 2019', 'Microsoft Windows Server 2019 Standard':'Windows Server 2019', 'Майкрософт Windows Server 2019 Standard':'Windows Server 2019', 
            
            'Windows Server 2022 (Server Core installation)':'Windows Server 2022', 'Windows Server 2022, 23H2 Edition (Server Core installation)':'Windows Server 2022', 
            
           }


In [2]:
msrc = connection.query_df("""Select distinct product_names, doc_xml_date,  kb, cve from msrc_data where cve not like 'ADV%'""")
print("Got msrc")
osquery_patches = connection.query_df('select distinct hostname, hotfix_id from dwh_osquery_software_patches where toMonth(timestamp) >= toMonth(now()) -1 ')
print("Got osquery_patches")
osquery_version = connection.query_df('Select * from dict_osquery_system_os_version')
print("Got osquery_version")

Got msrc
Got osquery_patches
Got osquery_version


In [3]:
osquery_patches['hotfix_id'] = osquery_patches['hotfix_id'].str.lstrip('KB').astype(int)
osquery_patches = osquery_patches.groupby(['hostname']).max().reset_index()

msrc['kb'] = msrc['kb'].astype(int)
# osquery_version['os_name'] = osquery_version['os_name'].replace(rep_dict)

msrc['product_names'] = msrc['product_names'].apply(lambda x: 'Windows 10' if x.startswith('Windows 10') else x)
msrc['product_names'] = msrc['product_names'].apply(lambda x: 'Windows Server 2008' if x.startswith('Windows Server 2008') else x)
msrc['product_names'] = msrc['product_names'].apply(lambda x: 'Windows 7' if x.startswith('Windows 7') else x)
msrc['product_names'] = msrc['product_names'].apply(lambda x: 'Windows 8' if x.startswith('Windows 8') else x)
msrc['product_names'] = msrc['product_names'].apply(lambda x: 'Windows Server 2012' if x.startswith('Windows Server 2012') else x)
msrc['product_names'] = msrc['product_names'].apply(lambda x: 'Windows 11' if x.startswith('Windows 11') else x)
msrc['product_names'] = msrc['product_names'].apply(lambda x: 'Windows Server 2016' if x.startswith('Windows Server 2016') else x)
msrc['product_names'] = msrc['product_names'].apply(lambda x: 'Windows Server 2019' if x.startswith('Windows Server 2019') else x)
msrc['product_names'] = msrc['product_names'].apply(lambda x: 'Windows Server 2022' if x.startswith('Windows Server 2022') else x)
msrc['product_names'] = msrc['product_names'].apply(lambda x: 'Windows Server' if x.startswith('Windows Server, version') else x)

osquery_version['os_name'] = osquery_version['os_name'].apply(lambda x:'Windows 10' if "Windows 10" in x else x)
osquery_version['os_name'] = osquery_version['os_name'].apply(lambda x:'Windows Server 2019' if "Windows Server 2019" in x else x)
osquery_version['os_name'] = osquery_version['os_name'].apply(lambda x:'Windows Server 2012' if "Windows Server 2012" in x else x)
osquery_version['os_name'] = osquery_version['os_name'].apply(lambda x:'Windows Server 2016' if "Windows Server 2016" in x else x)
osquery_version['os_name'] = osquery_version['os_name'].apply(lambda x:'Windows Server 2008' if "Windows Server 2008" in x else x)
osquery_version['os_name'] = osquery_version['os_name'].apply(lambda x:'Windows 7' if "Windows 7" in x else x)
osquery_version['os_name'] = osquery_version['os_name'].apply(lambda x:'Windows 11' if "Windows 11" in x else x)
osquery_version['os_name'] = osquery_version['os_name'].apply(lambda x:'Windows 8' if "Windows 8" in x else x)

msrc = msrc[msrc['product_names'].isin(allowed_val)]
# osquery_version = osquery_version[osquery_version['os_name'].isin(allowed_val)]

msrc = msrc.sort_values(by = ['product_names', 'kb'])

print(msrc)

              product_names doc_xml_date       kb             cve
17095            Windows 10     2020-Jan  4528760   CVE-2020-0601
17096            Windows 10     2020-Jan  4528760   CVE-2020-0601
17097            Windows 10     2020-Jan  4528760   CVE-2020-0601
17099            Windows 10     2020-Jan  4528760   CVE-2020-0601
17100            Windows 10     2020-Jan  4528760   CVE-2020-0601
...                     ...          ...      ...             ...
17022   Windows Server 2022     2024-Oct  5044288  CVE-2024-43611
17067   Windows Server 2022     2024-Oct  5044288  CVE-2024-43615
17082   Windows Server 2022     2024-Oct  5044288   CVE-2024-6197
73594   Windows Server 2022     2024-Aug  5044288  CVE-2024-38202
149427  Windows Server 2022     2024-Apr  5044288   CVE-2022-0001

[117639 rows x 4 columns]


In [4]:
# res = (msrc.groupby('product_names')
#        .apply(lambda x: sorted(x[['kb', 'doc_initial_relise_date']].drop_duplicates().assign(doc_initial_relise_date = x['doc_initial_relise_date'].dt.strftime('%Y-%m-%d')).values.tolist(), key = lambda y: y[1]))
#        .reset_index(name = "kb_arr"))
res = (msrc.groupby('product_names')
       .apply(lambda x: sorted(x[['kb']].drop_duplicates().assign().values.tolist()))
       .reset_index(name = "kb_arr"))

msrc = msrc.sort_values(by = 'kb')
print(len(res['kb_arr'][0]))

302


In [5]:

# print(osquery_patches.loc[osquery_patches['hostname'] == 'n8c8caaeec211','hotfix_id'].values[0])
osquery_version = pd.merge(osquery_version, res, left_on = 'os_name', right_on = 'product_names', how = 'left')
osquery_version = pd.merge(osquery_version, osquery_patches, on = 'hostname', how = 'left')


def trim_array(row):
    arr = row['kb_arr']
    kb = row['hotfix_id']

    if not isinstance(arr, list) : return []
    start_from = next((i for i, x in enumerate(arr) if int(x[0]) == kb), None)
    
    if start_from is None:
        start_from = next((i for i, x in enumerate(arr) if int(x[0]) > kb), len(arr))
    
    return arr[start_from:]

osquery_version['kb_arr'] = osquery_version.apply(trim_array, axis = 1)


def get_cve(row):
    if row[ 'kb_arr'] == []: return []
        
    return msrc[(msrc['kb'] >= row['hotfix_id']) & (msrc['product_names'] == row['os_name'])]['cve'].unique()
    
osquery_version['cve'] = osquery_version.apply(get_cve, axis = 1)


                     timestamp   valid_to  \
0    2024-10-25 15:43:42+03:00 2024-10-25   
1    2024-03-11 12:04:09+03:00 2024-03-11   
2    2024-10-25 15:15:03+03:00 2024-10-25   
3    2024-10-25 15:42:45+03:00 2024-10-25   
4    2024-10-25 15:37:31+03:00 2024-10-25   
...                        ...        ...   
5253 2024-10-25 15:38:00+03:00 2024-10-25   
5254 2024-09-16 11:35:10+03:00 2024-09-16   
5255 2024-05-16 16:12:31+03:00 2024-05-16   
5256 2024-03-20 17:47:19+03:00 2024-03-20   
5257 2024-02-13 15:27:55+03:00 2024-02-13   

                                      uuid  \
0     0001c5b3-0b78-6745-8a10-58fcba583236   
1     0029784c-2895-11b2-a85c-b326d18e3a32   
2     006c824c-2ae6-11b2-a85c-ca878ccd0095   
3     006e3f42-bbe9-20f2-b335-bc41133229b4   
4     007d9862-682e-403d-9a58-f0e521697b1a   
...                                    ...   
5253  ffdcad4c-2893-11b2-a85c-da1689353b3a   
5254  ffdf1a42-e500-9667-40f1-76238460cae4   
5255  fffd1a42-3d54-bac5-eec1-dace7cf372c0   

In [7]:
print(msrc)

print(msrc[(msrc['kb'] > 5043126) & (msrc['product_names'] == 'Windows 10')]['kb'].unique())
print(osquery_version['cve'][0])

             product_names doc_xml_date       kb             cve
17095           Windows 10     2020-Jan  4528760   CVE-2020-0601
19781           Windows 10     2020-Jan  4528760   CVE-2020-0640
19783           Windows 10     2020-Jan  4528760   CVE-2020-0640
19784           Windows 10     2020-Jan  4528760   CVE-2020-0640
19785           Windows 10     2020-Jan  4528760   CVE-2020-0640
...                    ...          ...      ...             ...
16231  Windows Server 2008     2024-Oct  5044356  CVE-2024-43556
16473  Windows Server 2008     2024-Oct  5044356  CVE-2024-43564
16474  Windows Server 2008     2024-Oct  5044356  CVE-2024-43564
16019  Windows Server 2008     2024-Oct  5044356  CVE-2024-43549
16899  Windows Server 2008     2024-Oct  5044356  CVE-2024-43593

[117639 rows x 4 columns]
[5044273 5044277 5044286 5044293]
<StringArray>
['CVE-2024-20659', 'CVE-2024-43554', 'CVE-2024-43553', 'CVE-2024-43551',
 'CVE-2024-43550', 'CVE-2024-43547', 'CVE-2024-43555', 'CVE-2024-43558',